In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import dolfinx as dfx

import h5py

from dolfinx.fem.petsc import NonlinearProblem

%matplotlib widget
from matplotlib import pyplot as plt
# plt.style.use('fivethirtyeight')

from mpi4py import MPI

import numpy as np

import os

from pathlib import Path

import scipy as sp

import ufl

from pyMoBiMP.cahn_hilliard_utils import (
    cahn_hilliard_form,
    charge_discharge_stop, 
    AnalyzeOCP,
    y_of_c,
    c_of_y,
    populate_initial_data)

from pyMoBiMP.fenicsx_utils import (evaluation_points_and_cells,
                           get_mesh_spacing,
                           time_stepping,
                           NewtonSolver,
                           FileOutput,
                           Fenicx1DOutput)

from pyMoBiMP.gmsh_utils import dfx_spherical_mesh

from pyMoBiMP.plotting_utils import (
    add_arrow, 
    plot_charging_cycle, 
    plot_time_sequence,
    PyvistaAnimation,
    animate_time_series)

comm_world = MPI.COMM_WORLD

In [ ]:
# Discretization
# --------------

# Set up the mesh
mesh_filename = "../Meshes/line_mesh.xdmf"

if os.path.isfile(mesh_filename):
    # Load mesh from file
    with dfx.io.XDMFFile(comm_world, mesh_filename, 'r') as file:
        mesh = file.read_mesh(name="Grid")
    print("load external mesh.")
else:
    n_elem = 32
    mesh = dfx.mesh.create_unit_interval(comm_world, n_elem)
    print("create mesh.")

dx_cell = get_mesh_spacing(mesh)

print(f"Cell spacing: h = {dx_cell}")

# For later plotting use
x = np.linspace(0, 1, 101)
points_on_proc, cells = evaluation_points_and_cells(mesh, x)

# Initial timestep size
dt = dfx.fem.Constant(mesh, dx_cell * 0.01)

In [ ]:
elem1 = ufl.FiniteElement("Lagrange", mesh.ufl_cell(), 1)

mixed_element = elem1 * elem1

V = dfx.fem.FunctionSpace(mesh, mixed_element)  # A mixed two-component function space

In [ ]:
# The mixed-element functions
u = dfx.fem.Function(V)
u0 = dfx.fem.Function(V)

In [ ]:
# Compute the chemical potential df/dc
a = 6. / 4
b = 0.2
cc = 5

# a = 5. # 6. / 4
# b = 0. # 0.2
# cc = 0 # 5

eps = 1e-3

free_energy = lambda u, log, sin: u * log(u) + (1-u) * log(1-u) + a * u * (1 - u) + b * sin(cc * np.pi * u)

res_c_left = sp.optimize.minimize_scalar(
    lambda c: free_energy(c, np.log, np.sin),
    bracket=(2 * eps, 0.05),
    bounds=(eps, 0.05))

assert res_c_left.success

c_left = res_c_left.x

res_c_right = sp.optimize.minimize_scalar(
    lambda c: free_energy(c, np.log, np.sin),
    bracket=(0.95, 1 - 2 * eps),
    bounds=(0.95, 1 - eps))

assert res_c_right.success

c_right = res_c_right.x

fig, ax = plt.subplots()

eps = 1e-3

c_plot = np.linspace(eps, 1-eps, 200)

ax.plot(c_plot, free_energy(c_plot, np.log, np.sin))

ax.scatter(c_left, free_energy(c_left, np.log, np.sin))
ax.scatter(c_right, free_energy(c_right, np.log, np.sin))

ax.set_xlabel(r"$c$")
ax.set_ylabel(r"$F(c)$")

plt.show()

In [ ]:
# Experimental setup
# ------------------

# charging current
I_charge = dfx.fem.Constant(mesh, 1e-1)

T_final = 2. / I_charge.value  # ending time

coords = ufl.SpatialCoordinate(mesh)
r2 = ufl.dot(coords, coords)

y, mu = u.split()
c = c_of_y(y)

c_bc_form = dfx.fem.form(r2 * c * ufl.ds)

def experiment(t, u, I_charge, cell_voltage=None, **kwargs):

    return charge_discharge_stop(t, u, I_charge, c_bc_form, c_of_y = c_of_y, **kwargs)

event_params = dict(I_charge=I_charge, stop_on_full=False, stop_at_empty=False, cycling=False, logging=True)

In [ ]:
# The variational form
# --------------------

params = dict(I_charge=I_charge, grad_c_bc=lambda c: 0.1 * I_charge)

F = cahn_hilliard_form(
    u,
    u0,
    dt,
    free_energy=lambda c: free_energy(c, ufl.ln, ufl.sin),
    theta=1.0,
    c_of_y=c_of_y,
    M=lambda c: 1. * c * (1 - c),
    lam=1e-3,
    **params
)

In [ ]:
# Initial data
# ------------

u_ini = dfx.fem.Function(V)

# Constant
c_ini_fun = lambda x: eps * np.ones_like(x[0])

# Initial charge distribution.
# c_ini_fun = lambda x: eps + 0.5 * np.sin(np.pi * x[0])

populate_initial_data(u_ini, c_ini_fun, lambda c: free_energy(c, ufl.ln, ufl.sin))

W, _ = V.sub(0).collapse()

c = dfx.fem.Function(W)
c.interpolate(dfx.fem.Expression(c_of_y(u_ini.sub(0)), W.element.interpolation_points()))

plt.figure()

plt.plot(x, u_ini.sub(0).eval(points_on_proc, cells), label="y")
plt.plot(x, c.eval(points_on_proc, cells), label="c")
plt.plot(x, u_ini.sub(1).eval(points_on_proc, cells), label=r"$\mu$")

plt.legend()

plt.show()

In [ ]:
problem = NonlinearProblem(F, u)

solver = NewtonSolver(comm_world, problem)

In [ ]:
# Set up experiment
# -----------------

u.interpolate(u_ini)

n_out = 501

output_np = Fenicx1DOutput(u, np.linspace(0, T_final, n_out), x)

results_folder = Path("simulation_output")
results_folder.mkdir(exist_ok=True, parents=True)

filename = results_folder / "CH_4_min_1D.xdmf"
output_xdmf = FileOutput(u, np.linspace(0, T_final, n_out), filename)

rt_analysis = AnalyzeOCP(u, c_of_y=c_of_y, filename=results_folder / "CH_4_min_1D_rt.txt")

In [ ]:
# Run the experiment
# ------------------

time_stepping(
    solver,
    u,
    u0,
    T_final,
    dt,
    dt_max=1e-1,
    tol=1e-5,
    event_handler=experiment,
    output=(output_xdmf, output_np),
    runtime_analysis=rt_analysis,
    **event_params,
)

## Read back the data

In [ ]:
with h5py.File(results_folder / "CH_4_min_1D.h5", 'r') as f:
    print(f.keys())

    # grid coordinates
    x_data = f["Mesh/mesh/geometry"][()]

    # time steps (convert from string to float)
    t = [float(t.replace("_", ".")) for t in f["Function/y"].keys()]

    # list of data stored as numpy arrays
    u_data = [(f["Function/y"][u_key][()].squeeze(),
               f["Function/mu"][u_key][()].squeeze())
              for u_key in f["Function/y"]]

sorted_indx = np.argsort(t)

t = np.array(t)[sorted_indx]
u_data = np.array(u_data)[sorted_indx]

rt_data  = np.loadtxt(results_folder / "CH_4_min_1D_rt.txt")

In [ ]:
fig, ax = plot_time_sequence((x_data, t, u_data), lambda y: np.exp(y) / (1 + np.exp(y)))

plt.show()

In [ ]:
mesh_3d, _, _ = dfx_spherical_mesh(comm_world, resolution=0.1, optimize=False)

In [ ]:
chem_pot = lambda q: np.log(q / (1 - q)) + a * (1 - 2 * q) + b * np.pi * cc * np.cos(np.pi * cc * q)

anim = PyvistaAnimation(
    (x_data, t, u_data),
    rt_data=rt_data,
    meshes=[mesh_3d],
    c_of_y=lambda y: np.exp(y) / (1 + np.exp(y)),
    f_of_q=chem_pot,
    clim=[0.0, 1.0],
    cmap="fire",
)

widget = anim.get_slider_widget()

In [ ]:
q, f_bar, mu_bc = np.array(rt_analysis.data).T

chart = plot_charging_cycle([(r"$\mu|_{\partial\omega_I}$", q, mu_bc),
                             (rf"$\bar f_A$", q, f_bar)], lambda q: chem_pot(q))